### 实验一：手写数字识别
姓名：**石钧予** 学号：**2022E8013282111** 研究所：**计算技术研究所**

In [1]:
import os
import torch
import struct
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, Dataset
from Model.CustomLinear import CustomLinear
from Model.CustomReLU import CustomReLU
from Model.CustomLeakyReLU import CustomLeakyReLU
from Model.CustomConv2d import CustomConv2d
from Model.CustomFlatten import CustomFlatten
torch.cuda.empty_cache()

In [2]:
class MnistDataset(Dataset):
    def __init__(self, data_dir_path, data_type='train', dtype=torch.float32):
        if data_type == 'train':

            data = self._load_minist(os.path.join(
                data_dir_path, "train-images-idx3-ubyte"), 'data')
            label = self._load_minist(os.path.join(
                data_dir_path, "train-labels-idx1-ubyte"), 'label')
        elif data_type == 'test':

            data = self._load_minist(os.path.join(
                data_dir_path, "t10k-images-idx3-ubyte"), 'data')
            label = self._load_minist(os.path.join(
                data_dir_path, "t10k-labels-idx1-ubyte"), 'label')

        label = np.eye(10)[label.flatten()]
        self.data = torch.from_numpy(data.reshape(data.shape[0],1,28,28)).type(dtype)
        self.label = torch.from_numpy(label).type(dtype)

    def __getitem__(self, index):
        return self.data[index], self.label[index]

    def __len__(self):
        return len(self.data)

    def _load_minist(self, file_path, type='data'):
        binary_file = open(file_path, 'rb')
        binary_data = binary_file.read()
        binary_file.close()
        if type == 'data':
            fmt_header = '>iiii'
            _, num_images, num_rows, num_cols = struct.unpack_from(
                fmt_header, binary_data, 0)
        else:
            fmt_header = '>ii'
            num_rows, num_cols = 1, 1
            _, num_images = struct.unpack_from(fmt_header, binary_data, 0)
        data_size = num_images * num_rows * num_cols
        mat_data = struct.unpack_from(
            '>' + str(data_size) + 'B', binary_data, struct.calcsize(fmt_header))
        mat_data = np.reshape(mat_data, [num_images, num_rows * num_cols])
        return mat_data


In [3]:
class HandWritingClassification(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.cnn = nn.Sequential(
            
            CustomConv2d(1, 16, 3, 1, 1),
            nn.BatchNorm2d(16),
            CustomReLU(),

            CustomConv2d(16, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            CustomReLU(),
            
            CustomFlatten()
        )

        self.fc = nn.Sequential(
            CustomLinear(32*28*28, 1024),
            nn.BatchNorm1d(1024),
            CustomLeakyReLU(),
            
            CustomLinear(1024, 256),
            nn.BatchNorm1d(256),
            CustomLeakyReLU(),
            
            CustomLinear(256, 10),
        ) 

    def forward(self, input):
        return self.fc(self.cnn(input))


In [4]:
def train_loop(dataloader, model, loss_fn, optimizer, device='cuda'):
    size = len(dataloader.dataset)
    for idx_batch, (input_batch, label_batch) in enumerate(dataloader):
        input_batch = input_batch.to(device)
        label_batch = label_batch.to(device)
        pred = model(input_batch)
        loss = loss_fn(pred, label_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if idx_batch % 100 == 0:
            loss, current = loss.item(), (idx_batch + 1) * len(input_batch)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn, device='cuda'):
    size = len(dataloader.dataset)
    batch_size = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for input_batch, label_batch in dataloader:
            input_batch = input_batch.to(device)
            label_batch = label_batch.to(device)
            pred = model(input_batch)
            test_loss += loss_fn(pred, label_batch).item()
            correct += \
                (pred.argmax(1) == label_batch.argmax(1)).type(torch.float).sum().item()
    test_loss /= batch_size
    correct /= size
    print(
        f"Test Error: \n Accuracy:{(100*correct):>0.1f} % , Avg loss : {test_loss:>8f} \n")


In [5]:
MNIST_DIR = './data/MNIST/'
BATCH_SIZE = 200
LR = 1e-3
EPOCHS = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
train_set = MnistDataset(MNIST_DIR, 'train')
test_set = MnistDataset(MNIST_DIR, 'test')
train_loader = DataLoader(train_set, BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, BATCH_SIZE, shuffle=False)
model = HandWritingClassification().to(device=DEVICE)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), LR)
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    model.train()
    train_loop(train_loader, model, loss_fn, optimizer)
    print()
    model.eval()
    test_loop(test_loader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.495348  [  200/60000]
loss: 0.061936  [20200/60000]
loss: 0.065573  [40200/60000]

Test Error: 
 Accuracy:98.5 % , Avg loss : 0.045585 

Epoch 2
-------------------------------
loss: 0.061743  [  200/60000]
loss: 0.027270  [20200/60000]
loss: 0.019603  [40200/60000]

Test Error: 
 Accuracy:98.8 % , Avg loss : 0.038019 

Epoch 3
-------------------------------
loss: 0.007139  [  200/60000]
loss: 0.031975  [20200/60000]
loss: 0.006129  [40200/60000]

Test Error: 
 Accuracy:98.8 % , Avg loss : 0.040006 

Epoch 4
-------------------------------
loss: 0.006885  [  200/60000]
loss: 0.008907  [20200/60000]
loss: 0.003304  [40200/60000]

Test Error: 
 Accuracy:98.8 % , Avg loss : 0.038197 

Epoch 5
-------------------------------
loss: 0.006713  [  200/60000]
loss: 0.002299  [20200/60000]
loss: 0.000572  [40200/60000]

Test Error: 
 Accuracy:98.4 % , Avg loss : 0.048132 

Epoch 6
-------------------------------
loss: 0.001556  [  200/60000]
loss:

In [6]:
PATH = './HandWrittingClassification.pt'
torch.save(model.state_dict(), PATH)
model.load_state_dict(torch.load(PATH))
model.eval()

HandWritingClassification(
  (cnn): Sequential(
    (0): CustomConv2d()
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): CustomReLU()
    (3): CustomConv2d()
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): CustomReLU()
    (6): CustomFlatten()
  )
  (fc): Sequential(
    (0): CustomLinear()
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): CustomLeakyReLU()
    (3): CustomLinear()
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): CustomLeakyReLU()
    (6): CustomLinear()
  )
)